In [7]:
import pandas as pd

# coding: utf-8

# 내가 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import clean, getNouns, getVector
from clustering import addClusterNumber, getClusteredArticle
from summary import getSummaryArticle

In [9]:
# main

# 명사까지 추출한 파일 불러오기, 1400개 기사
# test_df는 article_df임

test_df = pd.read_csv("article.csv")
test_df['nouns'] = test_df['nouns'].apply(lambda x: eval(x))        # 명사 열을 다시 리스트 형식으로 변환

vector_list = getVector(test_df)         # 카테고리 별로 명사를 벡터화한 리스트

addClusterNumber(test_df, vector_list)   # 군집 번호 열 생성
cluster_counts_df = getClusteredArticle(test_df)      # 상위 군집 10개에 해당하는 기사 df, 군집 개수 카운트한 df

summary_article = getSummaryArticle(test_df, cluster_counts_df)


In [8]:
import cx_Oracle as cx
import pandas as pd

# category, title, content, url
id = "c##2201058"
pw = "p2201058"
url = "10.30.3.95:1521/orcl"

conn = cx.connect(id, pw, url)

def insert(summary_article):

    sql = """insert into news(news_id, cate_id, title, content, link, views)
             values(news_id_seq.nextval, :1, :2, :3, :4, 0)"""

    cur = conn.cursor()
    cur.executemany(sql, summary_article)

    cur.close()
    conn.commit()
    conn.close()

insert(summary_article.values.tolist())

In [ ]:
# 카테고리를 숫자로 바꾸기
# news_id 날짜 + 카테고리 번호 + 기사 순서(시퀀스)로 만들기